# Chapter 3: 线性回归实践

## 实验概述

在本实验中，你将使用 **Diabetes 数据集**来学习线性回归的实现和应用。Diabetes 数据集包含 442 个糖尿病患者的数据，每个患者有 10 个生理特征，目标是预测一年后疾病进展的定量测量值。

### 实验目标

1. 理解线性回归的基本原理
2. 实现梯度下降算法
3. 手动实现线性回归模型
4. 使用 scikit-learn 的线性回归模型
5. 进行特征标准化和正则化
6. 评估模型性能

### 数据集信息

- **样本数量**: 442
- **特征数量**: 10
- **特征**: 年龄、性别、BMI、血压、血清测量值等
- **目标**: 一年后疾病进展的定量测量值

## Part 1: 数据加载与探索

### 1.1 导入必要的库

In [ ]:
# ===== 预填充代码 =====
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

# 设置随机种子
np.random.seed(42)

print("库导入成功！")

### 1.2 加载 Diabetes 数据集

In [ ]:
# ===== 预填充代码 =====
# 加载数据集
diabetes = load_diabetes()
X = diabetes.data
y = diabetes.target
feature_names = diabetes.feature_names

print("数据集信息:")
print(f"样本数量: {X.shape[0]}")
print(f"特征数量: {X.shape[1]}")
print(f"\n特征名称:")
for i, name in enumerate(feature_names):
    print(f"  {i+1}. {name}")
print(f"\n目标值范围: [{y.min():.1f}, {y.max():.1f}]")
print(f"目标值均值: {y.mean():.2f}")

### 1.3 数据探索

**任务**: 创建 DataFrame 并查看数据统计信息

In [ ]:
# TODO: 创建 DataFrame
df = pd.DataFrame(X, columns=feature_names)

# TODO: 添加目标列 'progression'
df['progression'] = y

print("前5行数据:")
print(df.head())

print("\n数据描述性统计:")
# TODO: 显示描述性统计信息
# 提示: 使用 describe() 方法
print(df.describe())

### 1.4 数据可视化

**任务**: 绘制 BMI 与疾病进展的关系图

In [ ]:
plt.figure(figsize=(10, 6))

# TODO: 绘制散点图
# 提示: plt.scatter(x, y, alpha=0.5)
plt.scatter(df['bmi'], df['progression'], alpha=0.5)

# 添加回归线
# 暂时不添加，等后面实现后再添加

plt.xlabel('BMI (体重指数)')
plt.ylabel('疾病进展 (定量测量值)')
plt.title('BMI 与疾病进展的关系')
plt.grid(True, alpha=0.3)
plt.show()

### 1.5 划分训练集和测试集

In [ ]:
# ===== 预填充代码 =====
# 划分数据集 (80% 训练, 20% 测试)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)

print(f"训练集大小: {X_train.shape}")
print(f"测试集大小: {X_test.shape}")

## Part 2: 特征预处理

### 2.1 特征标准化

In [ ]:
# ===== 预填充代码 =====
# 创建标准化器
scaler = StandardScaler()

# 标准化训练集和测试集
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("标准化前的统计信息 (年龄特征):")
print(f"训练集均值: {X_train[:, 0].mean():.2f}, 标准差: {X_train[:, 0].std():.2f}")
print(f"测试集均值: {X_test[:, 0].mean():.2f}, 标准差: {X_test[:, 0].std():.2f}")

print("\n标准化后的统计信息 (年龄特征):")
print(f"训练集均值: {X_train_scaled[:, 0].mean():.2f}, 标准差: {X_train_scaled[:, 0].std():.2f}")
print(f"测试集均值: {X_test_scaled[:, 0].mean():.2f}, 标准差: {X_test_scaled[:, 0].std():.2f}")

## Part 3: 手动实现线性回归

### 3.1 实现线性回归模型

In [ ]:
def linear_regression(X, theta):
    """
    线性回归前向传播
    
    参数:
    X: 特征矩阵 (m, n+1)，包含截距项
    theta: 参数向量 (n+1,)
    
    返回:
    预测值 (m,)
    """
    # TODO: 计算预测值
    # 提示: 使用矩阵乘法 np.dot(X, theta)
    return np.dot(X, theta)

### 3.2 实现损失函数

In [ ]:
def compute_mse(X, y, theta):
    """
    计算均方误差损失
    
    参数:
    X: 特征矩阵
    y: 真实值
    theta: 参数
    
    返回:
    均方误差
    """
    # TODO 1: 计算预测值
    y_pred = linear_regression(X, theta)
    
    # TODO 2: 计算均方误差
    # MSE = (1/m) * sum((y_pred - y)^2)
    m = len(y)
    mse = (1/m) * np.sum((y_pred - y) ** 2)
    
    return mse

# 测试损失函数
X_train_with_intercept = np.c_[np.ones((X_train_scaled.shape[0], 1)), X_train_scaled]
initial_theta = np.zeros(X_train_scaled.shape[1] + 1)
test_mse = compute_mse(X_train_with_intercept, y_train, initial_theta)
print(f"初始损失函数值: {test_mse:.4f}")

### 3.3 实现梯度计算

In [ ]:
def compute_gradient(X, y, theta):
    """
    计算梯度
    
    参数:
    X: 特征矩阵 (m, n+1)
    y: 真实值 (m,)
    theta: 参数 (n+1,)
    
    返回:
    梯度 (n+1,)
    """
    m = len(y)
    
    # TODO 1: 计算预测值
    y_pred = linear_regression(X, theta)
    
    # TODO 2: 计算误差
    error = y_pred - y
    
    # TODO 3: 计算梯度
    # gradient = (1/m) * X^T @ (y_pred - y)
    gradient = (1/m) * np.dot(X.T, error)
    
    return gradient

# 测试梯度计算
test_gradient = compute_gradient(X_train_with_intercept, y_train, initial_theta)
print(f"梯度形状: {test_gradient.shape}")
print(f"梯度前5个值: {test_gradient[:5]}")

### 3.4 实现梯度下降

In [ ]:
def gradient_descent(X, y, theta, alpha, num_iterations):
    """
    梯度下降算法
    
    参数:
    X: 特征矩阵
    y: 真实值
    theta: 初始参数
    alpha: 学习率
    num_iterations: 迭代次数
    
    返回:
    优化后的参数和损失历史
    """
    # 添加截距项
    m = len(y)
    X_with_intercept = np.c_[np.ones((m, 1)), X]
    
    loss_history = []
    
    for i in range(num_iterations):
        # TODO 1: 计算当前损失
        current_loss = compute_mse(X_with_intercept, y, theta)
        loss_history.append(current_loss)
        
        # TODO 2: 计算梯度
        gradient = compute_gradient(X_with_intercept, y, theta)
        
        # TODO 3: 更新参数
        theta = theta - alpha * gradient
        
        # 每100次迭代打印一次损失
        if (i + 1) % 100 == 0:
            print(f"迭代 {i+1}/{num_iterations}, 损失: {current_loss:.4f}")
    
    return theta, loss_history

# 测试梯度下降
alpha = 0.01  # 学习率
num_iterations = 1000  # 迭代次数

initial_theta = np.zeros(X_train_scaled.shape[1] + 1)

print("开始梯度下降...")
theta, loss_history = gradient_descent(X_train_scaled, y_train, initial_theta, alpha, num_iterations)
print(f"\n优化后的参数:")
print(f"截距: {theta[0]:.4f}")
for i, coef in enumerate(theta[1:], 1):
    print(f"{feature_names[i-1]}: {coef:.4f}")

### 3.5 可视化损失曲线

In [ ]:
# ===== 预填充代码 =====
# 绘制损失曲线
plt.figure(figsize=(10, 6))
plt.plot(loss_history)
plt.xlabel('迭代次数')
plt.ylabel('损失 (MSE)')
plt.title('梯度下降损失曲线')
plt.grid(True, alpha=0.3)
plt.show()

## Part 4: 模型预测与评估

### 4.1 实现预测函数

In [ ]:
def predict(X, theta):
    """
    使用训练好的模型进行预测
    
    参数:
    X: 特征矩阵 (m, n)
    theta: 参数向量 (n+1,)
    
    返回:
    预测值 (m,)
    """
    # TODO: 添加截距项并计算预测值
    X_with_intercept = np.c_[np.ones((len(X), 1)), X]
    y_pred = linear_regression(X_with_intercept, theta)
    
    return y_pred

# 在测试集上进行预测
y_test_pred_manual = predict(X_test_scaled, theta)

# 计算均方误差
mse_manual = mean_squared_error(y_test, y_test_pred_manual)

    # 计算R²分数
r2_manual = r2_score(y_test, y_test_pred_manual)

print(f"手动实现的线性回归:")
print(f"  测试集 MSE: {mse_manual:.4f}")
print(f"  测试集 R²: {r2_manual:.4f}")

    # 绘制预测 vs 真实值
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_test_pred_manual, alpha=0.5)

    # 绘制完美预测线
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)

plt.xlabel('真实值')
plt.ylabel('预测值')
plt.title('真实值 vs 预测值')
plt.grid(True, alpha=0.3)
plt.show()

## Part 5: 使用 scikit-learn 的线性回归

### 5.1 创建并训练 scikit-learn 模型

In [ ]:
from sklearn.linear_model import LinearRegression

# TODO: 创建线性回归模型
sklearn_lr = LinearRegression()

# TODO: 训练模型
sklearn_lr.fit(X_train_scaled, y_train)

# TODO: 在测试集上预测
y_test_pred_sklearn = sklearn_lr.predict(X_test_scaled)

# 计算指标
mse_sklearn = mean_squared_error(y_test, y_test_pred_sklearn)
r2_sklearn = r2_score(y_test, y_test_pred_sklearn)

print(f"scikit-learn 线性回归:")
print(f"  测试集 MSE: {mse_sklearn:.4f}")
print(f"  测试集 R²: {r2_sklearn:.4f}")

# 比较两种方法的结果
print(f"\n两种方法比较:")
print(f"  MSE 差值: {abs(mse_manual - mse_sklearn):.4f}")
print(f"  R² 差值: {abs(r2_manual - r2_sklearn):.4f}")

# 显示 sklearn 的系数
print(f"\nsklearn 回归系数:")
print(f"截距: {sklearn_lr.intercept_:.4f}")
for i, coef in enumerate(sklearn_lr.coef_):
    print(f"{feature_names[i]}: {coef:.4f}")

## Part 6: 挑战练习

### 6.1 实现 L2 正则化（Ridge 回归）

In [ ]:
def ridge_regression_gradient_descent(X, y, theta, alpha, lambda_reg, num_iterations):
    """
    带L2正则化的梯度下降
    
    参数:
    lambda_reg: L2正则化系数
    """
    m = len(y)
    X_with_intercept = np.c_[np.ones((m, 1)), X]
    
    loss_history = []
    
    for i in range(num_iterations):
        # 计算当前损失
        y_pred = linear_regression(X_with_intercept, theta)
        mse = (1/(2*m)) * np.sum((y_pred - y) ** 2)
        # 添加L2正则项
        l2_reg = (lambda_reg / (2*m)) * np.sum(theta[1:] ** 2)
        total_loss = mse + l2_reg
        loss_history.append(total_loss)
        
        # 计算梯度
        gradient = (1/m) * X_with_intercept.T @ (y_pred - y)
        # 添加L2正则的梯度（不惩罚截距项）
        gradient[1:] += (lambda_reg/m) * theta[1:]
        
        # 更新参数
        theta = theta - alpha * gradient
    
    return theta, loss_history

# 尝试不同的正则化系数
lambda_values = [0.01, 0.1, 1.0, 10.0]
results = []

print("Ridge 回归正则化效果:")
for lambda_reg in lambda_values:
    theta_ridge, _ = ridge_regression_gradient_descent(
        X_train_scaled, y_train, initial_theta, alpha, lambda_reg, num_iterations
    )
    
    y_pred_ridge = predict(X_test_scaled, theta_ridge)
    mse_ridge = mean_squared_error(y_test, y_pred_ridge)
    
    results.append({
        'lambda': lambda_reg,
        'mse': mse_ridge
    })
    
    print(f"lambda={lambda_reg}: MSE = {mse_ridge:.4f}")

# 找到最佳正则化系数
best_result = min(results, key=lambda x: x['mse'])
print(f"\n最佳 lambda: {best_result['lambda']}, MSE: {best_result['mse']:.4f}")

### 6.2 特征重要性分析

In [ ]:
# 提取 sklearn 模型的系数（不包括截距）
feature_importance = np.abs(sklearn_lr.coef_)

# 创建 DataFrame
importance_df = pd.DataFrame({
    'feature': feature_names,
    'coefficient': sklearn_lr.coef_,
    'absolute_importance': feature_importance
}).sort_values('absolute_importance', ascending=False)

print("特征重要性（基于系数绝对值）:")
print(importance_df)

# 可视化
plt.figure(figsize=(12, 6))
plt.barh(importance_df['feature'], importance_df['coefficient'])
plt.xlabel('系数值')
plt.ylabel('特征')
plt.title('线性回归特征系数')
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

### 6.3 多项式特征扩展

In [ ]:
# ===== 预填充代码 =====
from sklearn.preprocessing import PolynomialFeatures

# 选择最重要的两个特征进行可视化
important_features = ['bmi', 'bp']
feature_indices = [i for i, name in enumerate(feature_names) if name in important_features]

# 创建多项式特征（2阶）
poly = PolynomialFeatures(degree=2, include_bias=False)
X_train_poly = poly.fit_transform(X_train_scaled[:, feature_indices])
X_test_poly = poly.transform(X_test_scaled[:, feature_indices])

# 训练多项式回归模型
poly_lr = LinearRegression()
poly_lr.fit(X_train_poly, y_train)

# 预测
y_pred_poly = poly_lr.predict(X_test_poly)
mse_poly = mean_squared_error(y_test, y_pred_poly)
r2_poly = r2_score(y_test, y_pred_poly)

print(f"多项式回归 (degree=2):")
print(f"  测试集 MSE: {mse_poly:.4f}")
print(f"  测试集 R²: {r2_poly:.4f}")

print(f"\n与线性回归比较:")
print(f"  MSE 改善: {mse_sklearn - mse_poly:.4f}")
print(f"  R² 改善: {r2_poly - r2_sklearn:.4f}")

# 可视化
plt.figure(figsize=(10, 6))
plt.scatter(y_test, y_pred_poly, alpha=0.5, label='多项式回归')
plt.scatter(y_test, y_test_pred_sklearn, alpha=0.5, label='线性回归')
plt.plot([y.min(), y.max()], [y.min(), y.max()], 'r--', lw=2)
plt.xlabel('真实值')
plt.ylabel('预测值')
plt.title('线性回归 vs 多项式回归')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 总结

恭喜你完成了线性回归实验！在本实验中，你学习了：

1. ✅ **线性回归原理**: 最小二乘法、梯度下降
2. ✅ **手动实现**: 从零实现梯度下降算法
3. ✅ **scikit-learn**: 使用 LinearRegression 类
4. ✅ **特征预处理**: 标准化的重要性
5. ✅ **模型评估**: MSE、R² 指标
6. ✅ **正则化**: L2 正则化（Ridge 回归）
7. ✅ **多项式特征**: 非线性扩展

### 关键要点

- **梯度下降**: 通过迭代更新参数来最小化损失函数
- **学习率**: α 值的选择影响收敛速度和稳定性
- **特征缩放**: 标准化使梯度下降更快收敛
- **正则化**: 防止过拟合，提高模型泛化能力
- **评估指标**: MSE 衡量误差，R² 衡量解释方差比例

### 进一步学习

- 尝试其他正则化方法（L1 正则化 - Lasso）
- 学习岭回归（Ridge）和弹性网络（ElasticNet）
- 探索随机梯度下降（SGD）的变体
- 学习多项式回归的最佳阶数选择